In [ ]:
from constants import constants as k

## Test LISA-like orbit for TIANGO

Note: units unless specified otherwise are (AU, solar mass, day).

In [ ]:
# Test lisalike orbits

from lisalike import lisalike
ll = lisalike(triangleSideLength=k.rTIANGOm/k.mPerAU)

print(ll.getEccentricAnomaly(0.0, 1))
print(ll.getEccentricAnomaly(180.0, 1))

In [ ]:
print(k.omegaEarthPerDay)
print(ll.eccentricity)
print(ll.sigma(1))
print(ll.sigma(2))
print(ll.sigma(3))

In [ ]:
# Print relative positions in AU
print(ll.relativePosition(-20.0, 1))
print(ll.relativeVelocity(-20.0, 1))

## Test numerical integration of a LISA-like orbit for TIANGO (sun only)

In [ ]:
from orbit import orbit
myOrbit = orbit(system="EarthTrailing_1Body")

In [ ]:
import numpy as np
startTime = 0.0 # days
endTime = 50.0 #days
dt0 = 0.1 #days
eps = 1.e-15 #days
a0 = np.array(np.zeros(9))
useControl = False
pidParams = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
stepper = 'dopri5'

In [ ]:
initialPosition1 = ll.relativePosition(-20.0, 1)
initialVelocity1 = ll.relativeVelocity(-20.0, 1)
initialPosition2 = ll.relativePosition(-20.0, 2)
initialVelocity2 = ll.relativeVelocity(-20.0, 2)
initialPosition3 = ll.relativePosition(-20.0, 3)
initialVelocity3 = ll.relativeVelocity(-20.0, 3)
f0 = np.array([initialPosition1[0], initialPosition1[1], initialPosition1[2],
               initialVelocity1[0], initialVelocity1[1], initialVelocity1[2],
               initialPosition2[0], initialPosition2[1], initialPosition2[2],
               initialVelocity2[0], initialVelocity2[1], initialVelocity2[2],
               initialPosition3[0], initialPosition3[1], initialPosition3[2],
               initialVelocity3[0], initialVelocity3[1], initialVelocity3[2]])

In [ ]:
# Numerically evolve the LISA-like initial data
times, fj, ak = myOrbit.solveOde(startTime, endTime, dt0, eps, f0, a0, useControl, pidParams=pidParams, stepper=stepper)

In [ ]:
# Evaluate analytically the LISA-like orbit
fjAnalyticList = []
for time in times:
    x1 = ll.relativePosition(time - 20.0, 1)
    v1 = ll.relativeVelocity(time - 20.0, 1)
    x2 = ll.relativePosition(time - 20.0, 2)
    v2 = ll.relativeVelocity(time - 20.0, 2)
    x3 = ll.relativePosition(time - 20.0, 3)
    v3 = ll.relativeVelocity(time - 20.0, 3)
    fjAnalyticList.append([x1[0], x1[1], x1[2], v1[0], v1[1], v1[2],
                           x2[0], x2[1], x2[2], v2[0], v2[1], v2[2],
                           x3[0], x3[1], x3[2], v3[0], v3[1], v3[2]])
fjAnalytic = np.array(fjAnalyticList)

In [ ]:
from matplotlib import pylab as plt
plt.clf()
plt.plot(times, fj[:,0], label='numerical', color='b')
plt.plot(times, fjAnalytic[:,0], label='analytic', color='r')
plt.xlabel("Time (day)")
plt.ylabel("$x_1(t)$")
plt.legend(loc='best')
plt.show()

## Derivation of equations of motion

** Equations **

We want to solve the ODE system

$$\frac{d^2 x^i}{dt^2} == \frac{F^i}{m}$$

where $m$ is the satellite mass, $F^i$ is the net force, and $\ddot{x}^i$ is the acceleration. Let's try to bring this into standard form.

$$\frac{dx}{dt} = v_x\\
\frac{dy}{dt} = v_y\\
\frac{dz}{dt} = v_z\\
\frac{dv_x}{dt} = \frac{F_{x,grav}}{m} + a_x\\
\frac{dv_y}{dt} = \frac{F_{y,grav}}{m} + a_y\\
\frac{dv_z}{dt} = \frac{F_{z,grav}}{m} + a_z\\
$$

Here $a_i$ are control accelerations that will hold the satellite position at a desired location (e.g., L1).

Newton's law of gravity says of objects $1,2,\ldots N$ are exerting an influence on the satellite, then the gravitational force per mass is

$$\frac{F_{i,grav}}{m} = -\sum_N \frac{G M_N (x_i - x_{N,i})}{\sum_j [(x_j-x_{N,j})(x_j-x_{N,j})]^{3/2}}
$$

To excellent approximation, the paths of the celestial objects affecting the satellite's motion are unchanged by the satellite, so we can compute $x_{N,i}$ analytically. E.g., for starters we can simply put Earth and Sun in circular orbits about their center of mass.

So our function will need to know the current time $t$, current positions of the satellite $x_i$, and constants such as $M_N$ and $G$. Then, it will return the gravitational forces acting on the object.

In practice, we want the satellite to be near a Lagrange point, whose motion is known. We should only solve for deviations from the Lagrange point's motion. This is where the equations get kind of messy.

$$x_i = x_{i,0} + \delta x_i\\
v_i = v_{i,0} + \delta v_i = \frac{dx_{i,0}}{dt} + \frac{\delta x_i}{dt}\\
\frac{d v_i}{dt} = \frac{d^2 x_{i,0}}{dt} + \frac{d^2 \delta x_i}{dt^2}
$$

so

$$\frac{d x_0}{dt} + \frac{d \delta x}{dt} = v_{x,0} + \delta v_x\\
\frac{d y_0}{dt} + \frac{d \delta y}{dt} = v_{y,0} + \delta v_y\\
\frac{d z_0}{dt} + \frac{d \delta z}{dt} = v_{z,0} + \delta v_z\\
\frac{dv_{x,0}}{dt} + \frac{d \delta v_x}{dt} = \frac{F_{x,grav}}{m} + a_x\\
\frac{dv_{y,0}}{dt} + \frac{d \delta v_y}{dt} = \frac{F_{y,grav}}{m} + a_y\\
\frac{dv_{z,0}}{dt} + \frac{d \delta v_z}{dt} = \frac{F_{z,grav}}{m} + a_z\\
$$

The force of gravity is a sum of terms, one per object (no self gravity):

$$\frac{F_{i,grav}}{m} = -\sum_N \frac{G M_N (x_{i} - x_{N,i})}{ [\sum_j(x_{j}-x_{N,j})(x_{j}-x_{N,j})]^{3/2}}
$$

Now, the velocity equations simplify, because $dx_0/dt = v_{x,0}$. 

So then we have

$$\frac{d \delta x}{dt} = \delta v_x\\
\frac{d \delta y}{dt} = \delta v_y\\
\frac{d \delta z}{dt} = \delta v_z\\
\frac{d \delta v_x}{dt} = -\frac{dv_{x,0}}{dt} + \frac{F_{x,grav}}{m} + a_x\\
\frac{d \delta v_y}{dt} = -\frac{dv_{y,0}}{dt} + \frac{F_{y,grav}}{m} + a_y\\
\frac{d \delta v_z}{dt} = -\frac{dv_{z,0}}{dt} + \frac{F_{z,grav}}{m} + a_z\\
$$

where

$$\frac{F_{i,grav}}{m} = -\sum_N \frac{G M_N (x_{i,0} + \delta x_i - x_{N,i})}{ [\sum_j (x_{j,0}+\delta x_j-x_{N,j})(x_{j,0}+\delta x_j-x_{N,j})]^{3/2}}
$$

that is

$$\frac{F_{x,grav}}{m} = -\sum_N \frac{G M_N (x_{0} + \delta x - x_{N})}{ r^{3/2}}\\
\frac{F_{y,grav}}{m} = -\sum_N \frac{G M_N (y_{0} + \delta y - y_{N})}{ r^{3/2}}\\
\frac{F_{z,grav}}{m} = -\sum_N \frac{G M_N (z_{0} + \delta z_i - z_{N})}{ r^{3/2}}\\
$$ where 

$$ r = \sqrt{(x_0+\delta x-x_N)^2+(y_0+\delta y-y_N)^2+(z_0 + \delta z-z_N)^2}$$

Note: I am neglecting the self gravity of the satellites. If I wish, I can include the interaction among the satellites by adding those terms to the sum. For 3 satellites, we'll simply need 3 copies of these equations, along with 3 control accelerations per satellite.

These equations are now in the standard form

$$\frac{d f_i}{dt} = g_i(f_j,t)$$.

So the game is this: knowing $f_i$ now, what is $f_i$ a bit later? Then rinse and repeat.

That is, we'd like to know df_i, since 

$$f_i(t+dt) = f_i(t) + df_i$$.

The simplest approximation for $df_i$ is Euler's method: 

$$df_i = dt\mbox{ }g_i(f_j(t),t)$$, 

which you can get by multiplying the original ODEs through by $dt$. Better is RK2:

$$k_{1i} = dt\mbox{ }g(f_j(t),t)$$
$$k_{2i} = dt\mbox{ }g(f_j(t)+(1/2)k_{1j},t+dt/2)$$
$$df_i = k_{2i}$$.

Even better is RK4:

$$k_{1i} = dt\mbox{ }g(f_j(t),t)$$
$$k_{2i} = dt\mbox{ }g(f_j(t)+k_{1j}/2,t+dt/2)$$
$$k_{3i} = dt\mbox{ }g(f_j(t)+k_{2j}/2,t+dt/2)$$
$$k_{4i} = dt\mbox{ }g(f_j(t)+k_{3j},t+dt)$$
$$df_i = k_{1i}/6+k_{2i}/3+k_{3i}/3+k_{4i}/6$$.

The general Runge-Kutta method can be described by a Butcher tableau. 

|$c_I$ | $a_{IJ}$|
|------|---------|
|      | $b_J$   |

Each row above the horizontal line is a right-hand-side (RHS) evaluation. The $I^{\rm th}$ RHS is evaluated at time $t+c_I dt$ and with RHS variables $f_i(t)+\sum_J a_{IJ} k_{iJ}$. The actual step is $df_i = \sum_J b_J k_{Ji}$.

So Euler's method has a tableau

|     |     |
|-----|-----|
| 0 |   |   |
|   |   | 1 |

RK2 has a tablaeu

|     |     |   |
|-----|-----|---|
| 0   |     |   |
| 1/2 | 1/2 |   |
|     | 0   | 1 |

and the RK4 tableau is

|     |     |     |     |   |
|-----|-----|-----|-----|---|
| 0   |     |     |     |   |
| 1/2 | 1/2 |     |     |   |
| 1/2 | 0   | 1/2 |     |   | 
| 1   | 0   | 0   | 1   |   |
|     | 1/6 | 1/3 | 1/3 | 1/6 |

We can have two rows of $b_J$, which combine the $k_{iJ}$ differently. The second-to-bottom row is the combination that gives the best accuracy, while the bottom row gives a lower accuracy step. Subtracting it from the full accuracy step gives an error estimate.

Example: Dormand-Prince 5:

|       |           |                |          |         |              |        |    |
|-------|-----------|----------------|----------|---------|--------------|--------|----|
|0      |           |                |          |         |              |        |    |
|1/5    |1/5        |                |          |         |              |        |    |
|3/10   |3/40       | 9/40           |          |         |              |        |    |
|4/5    |44/45      | −56/15         |32/9      |         |              |        |    |
|8/9    |19372/6561 | −25360/2187    |64448/6561| −212/729|              |        |    |
|1      |9017/3168  | −355/33        |46732/5247| 49/176  | −5103/18656  |        |    |
|1      |35/384     | 0              |500/1113  | 125/192 | −2187/6784   |11/84   |    | 
|       |35/384     | 0              |500/1113  | 125/192 | −2187/6784   |11/84   |0   |
|       |5179/57600 | 0              |7571/16695| 393/640 | −92097/339200|187/2100|1/40|
 


To use these methods, here are the ingredients we will need:
0. Definitions for physical constants
1. A choice for fixed step dt, starting time, ending time, initial conditions
2. A function that evaluates the RHS g_i(f_j,t)
3. A function that takes a time step
4. The main code sets initial conditions then loops over time steps.

f_j is passed as a numpy array, whose length is the number of equations we're solving (2, in this case).

Now, we want to extend the notebook in two ways:

1. PID controller to hold the satellites at fixed positions.
2. Adaptive time stepping.

We'll try PID control. Add a control response f(t) to the equations of motion. Then we'll update f(t) as we go.

We only need three control function per satelite, so I'm going to make f an array.

The control system will do the following:

a. Compute the control error (difference between desired and current angle $\theta$ in this example)

b. Compute the derivative and running integral of the control error, using simple approximations (endpoint rule for integrals, difference now - prev. /dt for derivatives).

c. Take a sum, weighted by coefficients called gains, of the error, derivative of the error, and integral of the error. The result is used to update $a(t)$ for the next step.

In principle, we could update the control system less or more often than we take time steps, but for now, they're the same.